<a href="https://colab.research.google.com/github/delvTom/Data-challenge-MS2A-2025/blob/main/notebooks/01baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

BASE =  "/content/drive/MyDrive/datachallenge_data"
DATA = f"{BASE}/data"
OUT = f"{BASE}/outputs/submissions"

import os
import pandas as pd

os.makedirs(OUT, exist_ok=True)

print("DATA:", DATA)
print("OUT :", OUT)
print("Fichiers ->", os.listdir(DATA))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DATA: /content/drive/MyDrive/datachallenge_data/data
OUT : /content/drive/MyDrive/datachallenge_data/outputs/submissions
Fichiers -> ['X_test.csv', 'X_train.csv', 'y_train.csv', 'sample_submission.csv']


In [5]:
X_train = pd.read_csv(f"{DATA}/X_train.csv")
y_train = pd.read_csv(f'{DATA}/y_train.csv')
X_test = pd.read_csv(f"{DATA}/X_test.csv")

print("X_train :", X_train.shape)
print("y_train :", y_train.shape)
print("X_test :", X_test.shape)

X_train : (202933, 14)
y_train : (202933, 24)
X_test : (134673, 14)


In [6]:

X = X_train.drop(columns=["ID"])
X_test_final = X_test.drop(columns=["ID"])


Y = y_train.drop(columns=["ID"])

print("X :", X.shape)
print("Y :", Y.shape)
print("X_test_final :", X_test_final.shape)



X : (202933, 13)
Y : (202933, 23)
X_test_final : (134673, 13)


In [13]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


model = RandomForestRegressor(n_estimators=5, max_depth=7, min_samples_split=0.01, min_samples_leaf=30)
model.fit(X, Y)

RandomForestRegressor(max_depth=7, min_samples_leaf=30, min_samples_split=0.01,
                      n_estimators=5)

In [16]:
# Test du modèle en splitant mes data test


from sklearn.model_selection import train_test_split
import numpy as np

X_tr, X_val, Y_tr, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)



In [17]:
def weighted_rmse(y_true, y_pred):
    weights = np.where(y_true >= 0.5, 1.2, 1.0)
    se = weights * (y_true - y_pred) ** 2
    return np.sqrt(se.mean())

# Exemple avec split

model.fit(X_tr, Y_tr)
pred_val = model.predict(X_val)
score = weighted_rmse(Y_val.values, pred_val)
print("Weighted RMSE (80/20 split):", score)


Weighted RMSE (80/20 split): 0.12239718851662176


In [22]:
print(X_test_final.head())
print(X_test_final.shape)

   Humidity       M12       M13       M14       M15        M4        M5  \
0  0.869916  0.021914 -0.015737  0.009041  0.015662 -0.139344 -0.041396   
1  0.904594 -0.100771 -0.040901  0.006137  0.013496  0.843534  0.501194   
2  0.688853  0.703585  1.029723  0.785505  0.293766 -0.536002 -0.166262   
3  0.392112  0.648665  0.628007  0.416822  0.065360 -0.309533 -0.398324   
4  0.618233  0.155158  0.285085  0.081453  0.031029 -0.801587 -0.709802   

         M6        M7         R        S1        S2        S3  
0 -0.006051  0.001638  1.008217  1.000922  1.003093  0.998643  
1  0.134963  0.024947  1.010621  0.987593  1.003104  1.009033  
2 -0.029509 -0.015855  1.075560  1.015945  1.038617  1.067383  
3 -0.269763 -0.269454  1.160165  1.084636  1.011883  1.008619  
4 -0.279257 -0.089210  1.051850  1.120134  1.003001  1.007571  
(134673, 13)


In [23]:
preds_test = model.predict(X_test_final)

In [24]:
import numpy as np

preds_test = np.clip(preds_test, 0.0, 1.0) # Force les valeurs a rester dans l'intervalle

In [25]:
import pandas as pd

target_cols = [c for c in y_train.columns if c != "ID"]

submission = pd.DataFrame(preds_test, columns=target_cols)
submission.insert(0, "ID", X_test["ID"])  # Rajoute le ID en 1er coll

sub_path = f"{OUT}/baseline_submission_rf.csv"
submission.to_csv(sub_path, index=False)
print("✅ Soumission créée :", sub_path)


✅ Soumission créée : /content/drive/MyDrive/datachallenge_data/outputs/submissions/baseline_submission_rf.csv
